In [1]:
ls -l

total 6568
drwxrwxrwx 1 root root    4096 Apr 21 20:28 ASL_GSLR_Code/
drwxrwxrwx 1 root root    4096 Apr 21 19:39 Google-Isolated-American-Sign-Language-Recognition-main/
-rwxrwxrwx 1 root root   87507 Apr 21 19:56 Step1_Feature_Extraction.ipynb*
-rwxrwxrwx 1 root root   86353 Apr 21 19:57 Step2_NormalizeAndFillNa.ipynb*
-rwxrwxrwx 1 root root   85771 Apr 21 19:57 Step3_PaddingAndTruncating.ipynb*
-rw-r--r-- 1 root root    9649 Apr 21 19:28 Untitled.ipynb
-rwxrwxrwx 1 root root    3352 Apr 21 19:41 sign_to_prediction_index_map.json*
-rwxrwxrwx 1 root root 6439210 Apr 21 19:41 train.csv*
drwxrwxrwx 1 root root    4096 Apr 21 20:11 train_landmark_files/


In [2]:
DATA_PATH = "train_landmark_files/"
TRAIN_PATH ="train.csv"
JSON_PATH =  "sign_to_prediction_index_map.json"

In [3]:
#!pip install pyarrow

In [4]:
import os
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm

In [13]:
train = pd.read_csv("train.csv")
train.head()


,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [14]:
class_counts = pd.DataFrame(train["sign"].value_counts())
class_counts

,count
sign,
listen,415
look,414
shhh,411
donkey,410
mouse,408
...,...
dance,312
person,312
beside,310


In [15]:
train_copy=train.copy()

In [16]:
train_copy.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


## creating "final_train.csv" file

In [24]:
train = pd.read_csv("train.csv")
train = train.drop(["participant_id","sequence_id"],axis=1)
import json
def read_json(path):
    with open(path, "r") as file:
        json_data = json.load(file)
    return json_data
s2p_map = read_json("sign_to_prediction_index_map.json")
p2s_map = {v: k for k, v in s2p_map.items()}

encoder = lambda x: s2p_map.get(x)
decoder = lambda x: p2s_map.get(x)

train["label"] =train["sign"].map(encoder)
train = train.drop(["sign"],axis=1)
train.to_csv("final_train.csv")

In [28]:
def preprocess_data(data, keep_indices):
    #dropping most of the face landmarks
    data = data.drop(["z"],axis=1)
    #keep_indices = [0, 9, 11, 13, 14, 17, 117, 118, 119, 199, 346, 347, 348]
    mask = (data['type'] == 'face') & (~data['landmark_index'].isin(keep_indices))
    data.drop(data[mask].index,inplace=True)
    #dropping unnecessary columns
    data = data.drop(["row_id","type","landmark_index"], axis=1)
    #pivoting data such that each frame can be represented as each row
    data['index'] = data.groupby('frame').cumcount()
    pivoted = data.pivot(index='frame', columns='index',values=['x','y'])
    new_cols=[name[0]+str(name[1]) for name in pivoted.columns]
    pivoted.columns=new_cols
    data = data.drop(['index'],axis=1)
    return pivoted

    

In [29]:
directories = []
for dirname, _, filenames in os.walk('train_landmark_files'):
    for filename in filenames:
        if filename.endswith(".parquet"):
            directories.append(os.path.join(dirname, filename))

In [30]:
len(directories)

94477

In [31]:

directories

['train_landmark_files/16069/100015657.parquet',
 'train_landmark_files/16069/1002113535.parquet',
 'train_landmark_files/16069/10042041.parquet',
 'train_landmark_files/16069/1004211348.parquet',
 'train_landmark_files/16069/100438640.parquet',
 'train_landmark_files/16069/1005009451.parquet',
 'train_landmark_files/16069/1005223850.parquet',
 'train_landmark_files/16069/1005492440.parquet',
 'train_landmark_files/16069/1006058791.parquet',
 'train_landmark_files/16069/1006443357.parquet',
 'train_landmark_files/16069/1007073382.parquet',
 'train_landmark_files/16069/1007127288.parquet',
 'train_landmark_files/16069/1007343357.parquet',
 'train_landmark_files/16069/1007589714.parquet',
 'train_landmark_files/16069/1008185618.parquet',
 'train_landmark_files/16069/1009615872.parquet',
 'train_landmark_files/16069/1010007890.parquet',
 'train_landmark_files/16069/1010611404.parquet',
 'train_landmark_files/16069/1011172246.parquet',
 'train_landmark_files/16069/1011655866.parquet',
 'tr

In [33]:
ls -l

total 11644
drwxrwxrwx 1 root root    4096 Apr 21 20:28 ASL_GSLR_Code/
drwxrwxrwx 1 root root    4096 Apr 21 19:39 Google-Isolated-American-Sign-Language-Recognition-main/
-rwxrwxrwx 1 root root   78187 Apr 21 20:53 Step1_Feature_Extraction.ipynb*
-rwxrwxrwx 1 root root   86353 Apr 21 19:57 Step2_NormalizeAndFillNa.ipynb*
-rwxrwxrwx 1 root root   85771 Apr 21 19:57 Step3_PaddingAndTruncating.ipynb*
-rw-r--r-- 1 root root    9649 Apr 21 19:28 Untitled.ipynb
-rw-r--r-- 1 root root 5205410 Apr 21 20:51 final_train.csv
-rwxrwxrwx 1 root root    3352 Apr 21 19:41 sign_to_prediction_index_map.json*
-rwxrwxrwx 1 root root 6439210 Apr 21 19:41 train.csv*
drwxrwxrwx 1 root root    4096 Apr 21 20:11 train_landmark_files/


In [34]:
input_dir = 'train_landmark_files'
output_dir = 'train_landmark_files_preprocessed'

keep_indices_random = [0, 9, 11, 13, 14, 17, 117, 118, 119, 199, 346, 347, 348]


for directory in tqdm(directories):
    # extract the common part of the file path
    path_parts = directory.split("/")
    frame_num = path_parts[-2]
    file_name = path_parts[-1]
    input_file_path = os.path.join(input_dir, frame_num, file_name)
    output_file_path = os.path.join(output_dir, frame_num, file_name)
    
    # read the parquet file into a pandas dataframe
    df = pd.read_parquet(directory)
    
    # preprocess the data
    new_data = preprocess_data(df, keep_indices_random)
    
    # save the data to a new directory
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    new_data.to_parquet(output_file_path)


100%|██████████| 94477/94477 [3:14:14<00:00,  8.11it/s]  
